## The Machine Learning Part with Daily Data

First import any lib, we will install Pandasql, statsmodels, and fbprophet via PyPl on Databricks

Python Lib Ref:  
Pandasql: https://pypi.org/project/pandasql/  
Statsmodel: https://www.statsmodels.org/stable/index.html

Time Series Ref:  
https://otexts.com/fpp2/regression.html  
https://www.machinelearningplus.com/time-series/time-series-analysis-python/  
https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/  
https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/

https://facebook.github.io/prophet/docs/quick_start.html#python-api

In [3]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#pip install -U pandasql
from pandasql import sqldf

#pip install -U statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose


#pip install fbprophet
from fbprophet import Prophet


import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j").setLevel(logging.ERROR)

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.

Now we load the data we previously prepared

In [5]:
df_MSC_SalesTemp = pd.read_csv("/dbfs/FileStore/tables/MSC_SalesTemp.csv")

df_MSC_SalesTemp['date'] =  pd.to_datetime(df_MSC_SalesTemp['date'])

df_MSC_SalesTemp.index = pd.to_datetime(df_MSC_SalesTemp.date)

print(df_MSC_SalesTemp.shape)
print(df_MSC_SalesTemp.head())
print(df_MSC_SalesTemp.dtypes)




(1034, 4)
 date DailyRev ItemsSold AvgTemp
date 
2013-01-01 2013-01-01 2.401532e+05 464.0 -1.833333
2013-01-02 2013-01-02 1.809636e+06 2999.0 -0.250000
2013-01-03 2013-01-03 1.517603e+06 2433.0 -1.958333
2013-01-04 2013-01-04 1.322663e+06 2234.0 -0.083333
2013-01-05 2013-01-05 1.339435e+06 2162.0 -1.833333
date datetime64[ns]
DailyRev float64
ItemsSold float64
AvgTemp float64
dtype: object

In [6]:
#Check any missing values

df_MSC_SalesTemp.isnull().sum()



Out[3]: date 0
DailyRev 0
ItemsSold 0
AvgTemp 0
dtype: int64

We should review the data by plotting them against the dates

In [8]:
def plot_df(x, y, title="", xlabel='', ylabel='', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    display()

plot_df(x=df_MSC_SalesTemp.date, y=df_MSC_SalesTemp.AvgTemp, xlabel='Date', ylabel='Temperature', title='Daily Temperature VS Dates') 

plot_df(x=df_MSC_SalesTemp.date, y=df_MSC_SalesTemp.DailyRev, xlabel='Date', ylabel='Revenue', title='Daily Sales Revenue VS Dates')

plot_df(x=df_MSC_SalesTemp.date, y=df_MSC_SalesTemp.ItemsSold, xlabel='Date', ylabel='Counts', title='Daily Sales Count VS Dates')

From above plots, we can observe that there may be some seasonality and trends. We will decompose latter below.

Next, we will do ADF test. With ADF, the null hypothesis is that a unit root is present in a time series sample. The alternative hypothesis is different depending on which version of the test is used, but is usually stationarity or trend-stationarity. It is an augmented version of the Dickey–Fuller test for a larger and more complicated set of time series models.

In [10]:
from statsmodels.tsa.stattools import adfuller

# ADF Test
result_adf_temp = adfuller(df_MSC_SalesTemp.AvgTemp.values, autolag='AIC')
print(f'ADF Statistic: {result_adf_temp[0]}')
print(f'p-value: {result_adf_temp[1]}')
for key, value in result_adf_temp[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')
    
    

/databricks/python/lib/python3.7/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping
/databricks/python/lib/python3.7/site-packages/statsmodels/base/wrapper.py:100: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
 defaults=argspec[3])
ADF Statistic: -2.8193306415113804
p-value: 0.055565136200463675
Critial Values:
 1%, -3.4367333690404767
Critial Values:
 5%, -2.8643583648001925
Critial Values:
 10%, -2.568270618452702

In [11]:
# ADF Test
result_adf_dr = adfuller(df_MSC_SalesTemp.DailyRev.values, autolag='AIC')
print(f'ADF Statistic: {result_adf_dr[0]}')
print(f'p-value: {result_adf_dr[1]}')
for key, value in result_adf_dr[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')



ADF Statistic: -4.265066529536093
p-value: 0.0005103274601376279
Critial Values:
 1%, -3.436834649927693
Critial Values:
 5%, -2.864403033735098
Critial Values:
 10%, -2.568294409927983

In [12]:
# ADF Test
result_adf_temp = adfuller(df_MSC_SalesTemp.ItemsSold.values, autolag='AIC')
print(f'ADF Statistic: {result_adf_temp[0]}')
print(f'p-value: {result_adf_temp[1]}')
for key, value in result_adf_temp[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')



ADF Statistic: -3.599689373733299
p-value: 0.0057669690363005175
Critial Values:
 1%, -3.436828225807217
Critial Values:
 5%, -2.8644002004847144
Critial Values:
 10%, -2.568292900881126

From above ADF test, only the daily temperature series has p-value slightly greater than 0.05, so will be the only one non-stationary, while the daily items count and revenue both stationary.

In this case, we will still do a decompose of all series.

In [14]:
# Additive Decomposition for Average Temp
result_temp_add = seasonal_decompose(df_MSC_SalesTemp['AvgTemp'], model='additive', freq=30)

pd.plotting.register_matplotlib_converters()
result_temp_add.plot()
display()

In [15]:
# Additive Decomposition for daily revenue
result_dr_add = seasonal_decompose(df_MSC_SalesTemp['DailyRev'], model='additive', freq=30)


result_dr_add.plot()
display()

In [16]:
# Multiplicative Decomposition for daily revenue
result_dr_mul = seasonal_decompose(df_MSC_SalesTemp['DailyRev'], model='multiplicative', freq = 30)

result_dr_mul.plot()
display()

In [17]:
# Additive Decomposition for # of Items Sold
result_items_add = seasonal_decompose(df_MSC_SalesTemp['ItemsSold'], model='additive', freq=30)


result_items_add.plot()
display()

In [18]:
# Multiplicative Decomposition for # of Items Sold
result_items_mu = seasonal_decompose(df_MSC_SalesTemp['ItemsSold'], model='multiplicative', freq=30)


result_items_mu.plot()
display()

We can see from above plots that for the daily sales rev and items count, the difference between additive and multiplicative modes is small. So we will just use the additive mode for all three columns of data.

Next, we will reconstruct the data set with different compoenents.

In [20]:
#print(result_temp_add.trend)
#print(result_temp_add.resid)



In [21]:
df_recons_temp_add = pd.concat([result_temp_add.seasonal, result_temp_add.trend, result_temp_add.resid, result_temp_add.observed], axis=1)
df_recons_temp_add.columns = ['seas', 'trend', 'resid', 'actual_values']

print(df_recons_temp_add.shape)
print(df_recons_temp_add.isnull().sum())
print(df_recons_temp_add)


(1034, 4)
seas 0
trend 30
resid 30
actual_values 0
dtype: int64
 seas trend resid actual_values
date 
2013-01-01 -0.556632 NaN NaN -1.833333
2013-01-02 -0.596755 NaN NaN -0.250000
2013-01-03 -0.560733 NaN NaN -1.958333
2013-01-04 -0.704576 NaN NaN -0.083333
2013-01-05 -0.321137 NaN NaN -1.833333
2013-01-06 0.073050 NaN NaN -5.083333
2013-01-07 0.128169 NaN NaN -5.500000
2013-01-08 0.233319 NaN NaN -8.291667
2013-01-09 -0.027868 NaN NaN -10.916667
2013-01-10 0.556265 NaN NaN -11.666667
2013-01-11 0.797612 NaN NaN -13.000000
2013-01-12 1.215585 NaN NaN -13.000000
2013-01-13 1.031304 NaN NaN -13.440000
2013-01-14 1.189799 NaN NaN -8.625000
2013-01-15 0.711756 NaN NaN -7.500000
2013-01-16 -0.039173 -9.382722 1.130228 -8.291667
2013-01-17 -0.236686 -9.432028 0.460381 -9.208333
2013-01-18 0.436909 -9.432722 -3.754187 -12.750000
2013-01-19 0.730840 -9.406333 -5.199507 -13.875000
2013-01-20 0.327913 -9.397306 -6.430607 -15.500000
2013-01-21 0.163485 -9.370222 -3.584929 -12.791667
2013-01-22 -0.616511 -9.259806 -5.082016 -14.958333
2013-01-23 -0.716863 -9.027167 -3.547637 -13.291667
2013-01-24 -0.749616 -8.738972 -1.594745 -11.083333
2013-01-25 -0.359366 -8.408417 -3.815551 -12.583333
2013-01-26 -0.476711 -8.008417 -5.723206 -14.208333
2013-01-27 -0.506617 -7.553556 -6.439827 -14.500000
2013-01-28 -0.255823 -7.099694 -5.519482 -12.875000
2013-01-29 -0.573513 -6.776389 0.099902 -7.250000
2013-01-30 -0.297423 -6.638194 2.143951 -4.791667
... ... ... ... ...
2015-10-02 0.711756 9.561900 1.034036 11.307692
2015-10-03 -0.039173 9.134190 2.957614 12.052632
2015-10-04 -0.236686 8.711571 -0.084641 8.390244
2015-10-05 0.436909 8.283133 2.450690 11.170732
2015-10-06 0.730840 7.728021 -3.580813 4.878049
2015-10-07 0.327913 7.190652 -7.375708 0.142857
2015-10-08 0.163485 6.776171 -5.969067 0.970588
2015-10-09 -0.616511 6.365930 -6.514124 -0.764706
2015-10-10 -0.716863 5.922196 -4.205333 1.000000
2015-10-11 -0.749616 5.471698 -2.795252 1.926829
2015-10-12 -0.359366 5.011456 -0.895992 3.756098
2015-10-13 -0.476711 4.579989 -0.322028 3.781250
2015-10-14 -0.506617 4.287769 -1.326606 2.454545
2015-10-15 -0.255823 4.052612 1.667497 5.464286
2015-10-16 -0.573513 3.768482 0.605031 3.800000
2015-10-17 -0.297423 NaN NaN 4.365854
2015-10-18 -0.556632 NaN NaN 6.571429
2015-10-19 -0.596755 NaN NaN 5.071429
2015-10-20 -0.560733 NaN NaN 0.411765
2015-10-21 -0.704576 NaN NaN -1.058824
2015-10-22 -0.321137 NaN NaN 0.966667
2015-10-23 0.073050 NaN NaN 2.088235
2015-10-24 0.128169 NaN NaN 4.048780
2015-10-25 0.233319 NaN NaN 5.285714
2015-10-26 -0.027868 NaN NaN 5.428571
2015-10-27 0.556265 NaN NaN 4.540541
2015-10-28 0.797612 NaN NaN 1.128205
2015-10-29 1.215585 NaN NaN 0.151515
2015-10-30 1.031304 NaN NaN 0.529412
2015-10-31 1.189799 NaN NaN -1.375000

[1034 rows x 4 columns]

In [22]:
df_recons_dr_add = pd.concat([result_dr_add.seasonal, result_dr_add.trend, result_dr_add.resid, result_dr_add.observed], axis=1)
df_recons_dr_add.columns = ['seas', 'trend', 'resid', 'actual_values']

print(df_recons_dr_add.shape)
print(df_recons_dr_add.isnull().sum())
print(df_recons_dr_add)



(1034, 4)
seas 0
trend 30
resid 30
actual_values 0
dtype: int64
 seas trend resid actual_values
date 
2013-01-01 26967.714576 NaN NaN 2.401532e+05
2013-01-02 56122.973507 NaN NaN 1.809636e+06
2013-01-03 37494.880217 NaN NaN 1.517603e+06
2013-01-04 92043.175439 NaN NaN 1.322663e+06
2013-01-05 57799.451968 NaN NaN 1.339435e+06
2013-01-06 1793.891715 NaN NaN 1.368580e+06
2013-01-07 33707.565731 NaN NaN 1.074601e+06
2013-01-08 27375.203955 NaN NaN 1.067788e+06
2013-01-09 40251.810843 NaN NaN 7.547786e+05
2013-01-10 33218.355100 NaN NaN 7.569279e+05
2013-01-11 -31603.153297 NaN NaN 1.029180e+06
2013-01-12 19173.536520 NaN NaN 1.164855e+06
2013-01-13 -18577.320502 NaN NaN 9.381733e+05
2013-01-14 -45685.044659 NaN NaN 6.497296e+05
2013-01-15 -32615.348228 NaN NaN 7.480556e+05
2013-01-16 -19758.861431 939780.949711 -204461.463280 7.155606e+05
2013-01-17 -28778.255681 930680.218337 -215887.712656 6.860142e+05
2013-01-18 -10285.419054 908608.872610 -63899.061890 8.344244e+05
2013-01-19 -79485.869393 895789.271249 362924.438144 1.179228e+06
2013-01-20 -73998.357822 875279.811304 293858.879851 1.095140e+06
2013-01-21 -63441.555478 849516.490872 -169705.185394 6.163698e+05
2013-01-22 -31818.270843 831302.409263 -136905.835389 6.625783e+05
2013-01-23 -22019.930011 821121.020723 -132552.840712 6.665482e+05
2013-01-24 -13855.639264 822234.053664 -132319.653289 6.760588e+05
2013-01-25 -10683.260655 837048.974886 149117.155769 9.754829e+05
2013-01-26 12603.381014 849557.328316 386092.065670 1.248253e+06
2013-01-27 -2050.248594 844810.667020 199188.531574 1.041949e+06
2013-01-28 -8444.680841 836776.358068 -221054.687227 6.072770e+05
2013-01-29 39673.475318 838819.218716 -295433.539034 5.830592e+05
2013-01-30 14875.799849 845927.049329 -227154.599178 6.336482e+05
... ... ... ... ...
2015-10-02 -32615.348228 616084.923288 299237.293403 8.827069e+05
2015-10-03 -19758.861431 614985.579240 663339.249036 1.258566e+06
2015-10-04 -28778.255681 609737.059835 426561.479287 1.007520e+06
2015-10-05 -10285.419054 609588.086200 -149580.667147 4.497220e+05
2015-10-06 -79485.869393 614851.024344 -181081.404951 3.542838e+05
2015-10-07 -73998.357822 617099.089571 109746.553965 6.528473e+05
2015-10-08 -63441.555478 625138.241490 -3953.536012 5.577432e+05
2015-10-09 -31818.270843 640075.361996 82929.908847 6.911870e+05
2015-10-10 -22019.930011 647610.124925 152875.055086 7.784652e+05
2015-10-11 -13855.639264 645462.128313 -4344.572382 6.272619e+05
2015-10-12 -10683.260655 640549.083527 -201955.489539 4.279103e+05
2015-10-13 12603.381014 639840.565650 -251599.176155 4.008448e+05
2015-10-14 -2050.248594 639821.431107 -196651.093380 4.411201e+05
2015-10-15 -8444.680841 643045.794509 -249020.374452 3.855807e+05
2015-10-16 39673.475318 648449.093661 -2031.068979 6.860915e+05
2015-10-17 14875.799849 NaN NaN 7.062577e+05
2015-10-18 26967.714576 NaN NaN 5.400470e+05
2015-10-19 56122.973507 NaN NaN 4.298047e+05
2015-10-20 37494.880217 NaN NaN 6.681265e+05
2015-10-21 92043.175439 NaN NaN 5.550158e+05
2015-10-22 57799.451968 NaN NaN 5.755465e+05
2015-10-23 1793.891715 NaN NaN 9.626206e+05
2015-10-24 33707.565731 NaN NaN 9.814669e+05
2015-10-25 27375.203955 NaN NaN 7.941041e+05
2015-10-26 40251.810843 NaN NaN 5.919920e+05
2015-10-27 33218.355100 NaN NaN 4.912220e+05
2015-10-28 -31603.153297 NaN NaN 4.517047e+05
2015-10-29 19173.536520 NaN NaN 5.294738e+05
2015-10-30 -18577.320502 NaN NaN 7.930866e+05
2015-10-31 -45685.044659 NaN NaN 8.036176e+05

[1034 rows x 4 columns]

In [23]:
df_recons_items_add = pd.concat([result_items_add.seasonal, result_items_add.trend, result_items_add.resid, result_items_add.observed], axis=1)
df_recons_items_add.columns = ['seas', 'trend', 'resid', 'actual_values']

print(df_recons_items_add.shape)
print(df_recons_items_add.isnull().sum())
print(df_recons_items_add)



(1034, 4)
seas 0
trend 30
resid 30
actual_values 0
dtype: int64
 seas trend resid actual_values
date 
2013-01-01 29.813416 NaN NaN 464.0
2013-01-02 104.067456 NaN NaN 2999.0
2013-01-03 71.012910 NaN NaN 2433.0
2013-01-04 141.444224 NaN NaN 2234.0
2013-01-05 60.790688 NaN NaN 2162.0
2013-01-06 35.809375 NaN NaN 2129.0
2013-01-07 49.932607 NaN NaN 1811.0
2013-01-08 23.925032 NaN NaN 1709.0
2013-01-09 65.675032 NaN NaN 1114.0
2013-01-10 13.712405 NaN NaN 1229.0
2013-01-11 -16.672443 NaN NaN 1497.0
2013-01-12 36.077052 NaN NaN 1975.0
2013-01-13 -11.590625 NaN NaN 1651.0
2013-01-14 -77.254766 NaN NaN 914.0
2013-01-15 -39.916382 NaN NaN 1249.0
2013-01-16 -73.177241 1556.766667 -451.589426 1032.0
2013-01-17 -35.567927 1544.650000 -374.082073 1135.0
2013-01-18 -52.306163 1515.566667 -93.260504 1370.0
2013-01-19 -109.569888 1502.416667 730.153221 2123.0
2013-01-20 -30.155672 1474.083333 342.072339 1786.0
2013-01-21 -96.020378 1437.150000 -407.129622 934.0
2013-01-22 -38.520868 1410.466667 -342.945798 1029.0
2013-01-23 -36.024300 1395.300000 -267.275700 1092.0
2013-01-24 -18.273319 1402.016667 -156.743347 1227.0
2013-01-25 -21.119888 1432.216667 315.903221 1727.0
2013-01-26 -12.422339 1461.533333 906.889006 2356.0
2013-01-27 1.931582 1459.416667 403.651751 1865.0
2013-01-28 -16.148319 1442.883333 -455.735014 971.0
2013-01-29 51.924720 1445.000000 -498.924720 998.0
2013-01-30 -1.375978 1457.550000 -296.174022 1160.0
... ... ... ... ...
2015-10-02 -39.916382 784.183333 537.733049 1282.0
2015-10-03 -73.177241 785.766667 1111.410574 1824.0
2015-10-04 -35.567927 781.233333 834.334594 1580.0
2015-10-05 -52.306163 777.516667 -292.210504 433.0
2015-10-06 -109.569888 780.683333 -205.113445 466.0
2015-10-07 -30.155672 774.800000 -27.644328 717.0
2015-10-08 -96.020378 772.383333 -90.362955 586.0
2015-10-09 -38.520868 785.350000 26.170868 773.0
2015-10-10 -36.024300 791.333333 127.690967 883.0
2015-10-11 -18.273319 784.416667 24.856653 791.0
2015-10-12 -21.119888 772.916667 -282.796779 469.0
2015-10-13 -12.422339 766.933333 81.489006 836.0
2015-10-14 1.931582 761.916667 -122.848249 641.0
2015-10-15 -16.148319 760.800000 -167.651681 577.0
2015-10-16 51.924720 766.600000 -91.524720 727.0
2015-10-17 -1.375978 NaN NaN 821.0
2015-10-18 29.813416 NaN NaN 685.0
2015-10-19 104.067456 NaN NaN 455.0
2015-10-20 71.012910 NaN NaN 603.0
2015-10-21 141.444224 NaN NaN 642.0
2015-10-22 60.790688 NaN NaN 569.0
2015-10-23 35.809375 NaN NaN 1106.0
2015-10-24 49.932607 NaN NaN 1035.0
2015-10-25 23.925032 NaN NaN 812.0
2015-10-26 65.675032 NaN NaN 419.0
2015-10-27 13.712405 NaN NaN 471.0
2015-10-28 -16.672443 NaN NaN 412.0
2015-10-29 36.077052 NaN NaN 598.0
2015-10-30 -11.590625 NaN NaN 911.0
2015-10-31 -77.254766 NaN NaN 965.0

[1034 rows x 4 columns]

Then we will do the stationarity test on the residues for the three sets. With ADF, the null hypothesis is that a unit root is present in a time series sample. The alternative hypothesis is different depending on which version of the test is used, but is usually stationarity or trend-stationarity. It is an augmented version of the Dickey–Fuller test for a larger and more complicated set of time series models.

In [25]:
from statsmodels.tsa.stattools import adfuller

# ADF Test
result_adf_tempadd = adfuller(df_recons_temp_add.resid.values[15: 1019], autolag='AIC')
print(f'ADF Statistic: {result_adf_tempadd[0]}')
print(f'p-value: {result_adf_tempadd[1]}')
for key, value in result_adf_tempadd[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')




ADF Statistic: -10.43302520918883
p-value: 1.5924830558366232e-18
Critial Values:
 1%, -3.4370334797663844
Critial Values:
 5%, -2.8644907213150725
Critial Values:
 10%, -2.568341114581742

In [26]:


# ADF Test
result_adf_dradd = adfuller(df_recons_dr_add.resid.values[15: 1019], autolag='AIC')
print(f'ADF Statistic: {result_adf_dradd[0]}')
print(f'p-value: {result_adf_dradd[1]}')
for key, value in result_adf_dradd[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')





ADF Statistic: -10.26569160854379
p-value: 4.1250480698120405e-18
Critial Values:
 1%, -3.4370334797663844
Critial Values:
 5%, -2.8644907213150725
Critial Values:
 10%, -2.568341114581742

In [27]:


# ADF Test
result_adf_itemsadd = adfuller(df_recons_items_add.resid.values[15: 1019], autolag='AIC')
print(f'ADF Statistic: {result_adf_itemsadd[0]}')
print(f'p-value: {result_adf_itemsadd[1]}')
for key, value in result_adf_itemsadd[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')





ADF Statistic: -9.82529416244112
p-value: 5.207224095711075e-17
Critial Values:
 1%, -3.4370334797663844
Critial Values:
 5%, -2.8644907213150725
Critial Values:
 10%, -2.568341114581742

The p-value for above ADF test all much smaller than 0.05, so we can reject the Null Hypothesis of ADF, which means the residuals are stationary.

We should find the Co-variance of the temp against daily sales reve and items count.  
https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/

In [30]:
covariance_01 = np.cov(df_MSC_SalesTemp.AvgTemp, df_MSC_SalesTemp.DailyRev)

covariance_02 = np.cov(df_MSC_SalesTemp.AvgTemp, df_MSC_SalesTemp.ItemsSold)

print(covariance_01)
print(covariance_02)

[[ 1.07488190e+02 -1.58534986e+06]
 [-1.58534986e+06 1.49566151e+11]]
[[ 1.07488190e+02 -1.67572199e+03]
 [-1.67572199e+03 3.25726191e+05]]

We should try better methods, like Person Correcltion.  

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html

The Pearson correlation coefficient (named for Karl Pearson) can be used to summarize the strength of the linear relationship between two data samples.

The Pearson’s correlation coefficient is calculated as the covariance of the two variables divided by the product of the standard deviation of each data sample. It is the normalization of the covariance between the two variables to give an interpretable score.

In [32]:
# Pearson correlation coefficient for raw data
from scipy.stats import pearsonr

pcovariance_01 = pearsonr(df_MSC_SalesTemp.AvgTemp, df_MSC_SalesTemp.DailyRev)

pcovariance_02 = pearsonr(df_MSC_SalesTemp.AvgTemp, df_MSC_SalesTemp.ItemsSold)

print(pcovariance_01)
print(pcovariance_02)

(-0.3953921875694451, 5.005766994346229e-40)
(-0.28320130604859733, 1.5969103054508876e-20)

From above Pearson Correlation Coefficient, the pairs are unlikely to be linearly related, but we can still try to fit them and see what we get

In [34]:
from sklearn.linear_model import LinearRegression

# create X and y
feature_cols_01 = ['AvgTemp']

#print(data[feature_cols_01])

X_01 = df_MSC_SalesTemp[feature_cols_01]
y_01 = df_MSC_SalesTemp['DailyRev']
y_02 = df_MSC_SalesTemp['ItemsSold']

#Check if the features and labels sets are good
print(X_01)
print(y_01)

lm_01 = LinearRegression()
lm_01.fit(X_01, y_01)

lm_02 = LinearRegression()
lm_02.fit(X_01, y_02)


lm_01_intercept = lm_01.intercept_
# print intercept and coefficients
print ('Intercept for Daily Reve LM is:', lm_01_intercept)

lm_01_coef = lm_01.coef_
print ('Coeffcients for Daily Reve LM is:', lm_01.coef_)


lm_02_intercept = lm_02.intercept_
# print intercept and coefficients
print ('Intercept for Daily Iterm Count LM is:', lm_02_intercept)

lm_01_coef = lm_01.coef_
print ('Coeffcients for Daily Iterm Count LM is:', lm_02.coef_)


Rsq_01 = lm_01.score(X_01, y_01)
print('R-square for Daily Reve LM is:', Rsq_01)

Rsq_02 = lm_02.score(X_01, y_02)
print('R-square for Daily Iterm Count LM is:', Rsq_02)

AvgTemp
date 
2013-01-01 -1.833333
2013-01-02 -0.250000
2013-01-03 -1.958333
2013-01-04 -0.083333
2013-01-05 -1.833333
2013-01-06 -5.083333
2013-01-07 -5.500000
2013-01-08 -8.291667
2013-01-09 -10.916667
2013-01-10 -11.666667
2013-01-11 -13.000000
2013-01-12 -13.000000
2013-01-13 -13.440000
2013-01-14 -8.625000
2013-01-15 -7.500000
2013-01-16 -8.291667
2013-01-17 -9.208333
2013-01-18 -12.750000
2013-01-19 -13.875000
2013-01-20 -15.500000
2013-01-21 -12.791667
2013-01-22 -14.958333
2013-01-23 -13.291667
2013-01-24 -11.083333
2013-01-25 -12.583333
2013-01-26 -14.208333
2013-01-27 -14.500000
2013-01-28 -12.875000
2013-01-29 -7.250000
2013-01-30 -4.791667
... ...
2015-10-02 11.307692
2015-10-03 12.052632
2015-10-04 8.390244
2015-10-05 11.170732
2015-10-06 4.878049
2015-10-07 0.142857
2015-10-08 0.970588
2015-10-09 -0.764706
2015-10-10 1.000000
2015-10-11 1.926829
2015-10-12 3.756098
2015-10-13 3.781250
2015-10-14 2.454545
2015-10-15 5.464286
2015-10-16 3.800000
2015-10-17 4.365854
2015-10-18 6.571429
2015-10-19 5.071429
2015-10-20 0.411765
2015-10-21 -1.058824
2015-10-22 0.966667
2015-10-23 2.088235
2015-10-24 4.048780
2015-10-25 5.285714
2015-10-26 5.428571
2015-10-27 4.540541
2015-10-28 1.128205
2015-10-29 0.151515
2015-10-30 0.529412
2015-10-31 -1.375000

[1034 rows x 1 columns]
date
2013-01-01 2.401532e+05
2013-01-02 1.809636e+06
2013-01-03 1.517603e+06
2013-01-04 1.322663e+06
2013-01-05 1.339435e+06
2013-01-06 1.368580e+06
2013-01-07 1.074601e+06
2013-01-08 1.067788e+06
2013-01-09 7.547786e+05
2013-01-10 7.569279e+05
2013-01-11 1.029180e+06
2013-01-12 1.164855e+06
2013-01-13 9.381733e+05
2013-01-14 6.497296e+05
2013-01-15 7.480556e+05
2013-01-16 7.155606e+05
2013-01-17 6.860142e+05
2013-01-18 8.344244e+05
2013-01-19 1.179228e+06
2013-01-20 1.095140e+06
2013-01-21 6.163698e+05
2013-01-22 6.625783e+05
2013-01-23 6.665482e+05
2013-01-24 6.760588e+05
2013-01-25 9.754829e+05
2013-01-26 1.248253e+06
2013-01-27 1.041949e+06
2013-01-28 6.072770e+05
2013-01-29 5.830592e+05
2013-01-30 6.336482e+05
 ... 
2015-10-02 8.827069e+05
2015-10-03 1.258566e+06
2015-10-04 1.007520e+06
2015-10-05 4.497220e+05
2015-10-06 3.542838e+05
2015-10-07 6.528473e+05
2015-10-08 5.577432e+05
2015-10-09 6.911870e+05
2015-10-10 7.784652e+05
2015-10-11 6.272619e+05
2015-10-12 4.279103e+05
2015-10-13 4.008448e+05
2015-10-14 4.411201e+05
2015-10-15 3.855807e+05
2015-10-16 6.860915e+05
2015-10-17 7.062577e+05
2015-10-18 5.400470e+05
2015-10-19 4.298047e+05
2015-10-20 6.681265e+05
2015-10-21 5.550158e+05
2015-10-22 5.755465e+05
2015-10-23 9.626206e+05
2015-10-24 9.814669e+05
2015-10-25 7.941041e+05
2015-10-26 5.919920e+05
2015-10-27 4.912220e+05
2015-10-28 4.517047e+05
2015-10-29 5.294738e+05
2015-10-30 7.930866e+05
2015-10-31 8.036176e+05
Name: DailyRev, Length: 1034, dtype: float64
Intercept for Daily Reve LM is: 926938.0028877938
Coeffcients for Daily Reve LM is: [-14749.06085139]
Intercept for Daily Iterm Count LM is: 1342.7672657205048
Coeffcients for Daily Iterm Count LM is: [-15.58982416]
R-square for Daily Reve LM is: 0.15633498199095142
R-square for Daily Iterm Count LM is: 0.08020297974763113

In [35]:
from sklearn.linear_model import LinearRegression

# create X and y
feature_cols_02 = ['resid']

#print(data[feature_cols_01])

X_rec = df_recons_temp_add[feature_cols_02][15:1019]
y_rec01 = df_recons_dr_add['resid'][15:1019]
y_rec02 = df_recons_items_add['resid'][15:1019]

#Check if the features and labels sets are good
print(X_rec)
print(y_rec01)

lm_rec01 = LinearRegression()
lm_rec01.fit(X_rec, y_rec01)

lm_rec02 = LinearRegression()
lm_rec02.fit(X_rec, y_rec02)


lm_rec01_intercept = lm_rec01.intercept_
# print intercept and coefficients
print ('Intercept for Recons Daily Reve LM is:', lm_rec01_intercept)

lm_01_coef = lm_rec01.coef_
print ('Coeffcients for Recons Daily Reve LM is:', lm_rec01.coef_)


lm_rec02_intercept = lm_rec02.intercept_
# print intercept and coefficients
print ('Intercept for Daily Iterm Count LM is:', lm_rec02_intercept)

lm_rec02_coef = lm_rec02.coef_
print ('Coeffcients for Daily Iterm Count LM is:', lm_rec02.coef_)


Rsq_rec01 = lm_rec01.score(X_rec, y_rec01)
print('R-square for Daily Reve LM is:', Rsq_rec01)

Rsq_rec02 = lm_rec02.score(X_rec, y_rec02)
print('R-square for Daily Iterm Count LM is:', Rsq_rec02)

resid
date 
2013-01-16 1.130228
2013-01-17 0.460381
2013-01-18 -3.754187
2013-01-19 -5.199507
2013-01-20 -6.430607
2013-01-21 -3.584929
2013-01-22 -5.082016
2013-01-23 -3.547637
2013-01-24 -1.594745
2013-01-25 -3.815551
2013-01-26 -5.723206
2013-01-27 -6.439827
2013-01-28 -5.519482
2013-01-29 0.099902
2013-01-30 2.143951
2013-01-31 4.220521
2013-02-01 5.028005
2013-02-02 6.901011
2013-02-03 6.653187
2013-02-04 5.361415
2013-02-05 1.608327
2013-02-06 6.004556
2013-02-07 5.411739
2013-02-08 3.938204
2013-02-09 4.046432
2013-02-10 4.793974
2013-02-11 4.732250
2013-02-12 3.310977
2013-02-13 0.558037
2013-02-14 -2.098642
... ...
2015-09-17 3.315413
2015-09-18 7.427552
2015-09-19 4.172560
2015-09-20 1.830431
2015-09-21 6.033579
2015-09-22 1.596611
2015-09-23 1.758496
2015-09-24 4.734685
2015-09-25 7.542663
2015-09-26 7.354479
2015-09-27 7.674841
2015-09-28 1.039532
2015-09-29 -5.256041
2015-09-30 -2.770026
2015-10-01 -3.352443
2015-10-02 1.034036
2015-10-03 2.957614
2015-10-04 -0.084641
2015-10-05 2.450690
2015-10-06 -3.580813
2015-10-07 -7.375708
2015-10-08 -5.969067
2015-10-09 -6.514124
2015-10-10 -4.205333
2015-10-11 -2.795252
2015-10-12 -0.895992
2015-10-13 -0.322028
2015-10-14 -1.326606
2015-10-15 1.667497
2015-10-16 0.605031

[1004 rows x 1 columns]
date
2013-01-16 -204461.463280
2013-01-17 -215887.712656
2013-01-18 -63899.061890
2013-01-19 362924.438144
2013-01-20 293858.879851
2013-01-21 -169705.185394
2013-01-22 -136905.835389
2013-01-23 -132552.840712
2013-01-24 -132319.653289
2013-01-25 149117.155769
2013-01-26 386092.065670
2013-01-27 199188.531574
2013-01-28 -221054.687227
2013-01-29 -295433.539034
2013-01-30 -227154.599178
2013-01-31 -263332.658732
2013-02-01 -45007.893627
2013-02-02 190765.365178
2013-02-03 -27157.618960
2013-02-04 -452677.201047
2013-02-05 -193434.715983
2013-02-06 -251785.389525
2013-02-07 -70905.976445
2013-02-08 44114.040425
2013-02-09 382366.002369
2013-02-10 238453.604726
2013-02-11 -209828.294056
2013-02-12 -41702.365286
2013-02-13 -52959.218411
2013-02-14 76678.322360
 ... 
2015-09-17 41873.163429
2015-09-18 29088.853178
2015-09-19 -63502.517429
2015-09-20 -175907.749726
2015-09-21 -286754.776443
2015-09-22 -92680.072471
2015-09-23 -137454.875375
2015-09-24 -85817.962386
2015-09-25 99867.917711
2015-09-26 87168.181320
2015-09-27 -42271.955240
2015-09-28 -220770.023260
2015-09-29 -19584.661467
2015-09-30 -80091.251499
2015-10-01 192097.898173
2015-10-02 299237.293403
2015-10-03 663339.249036
2015-10-04 426561.479287
2015-10-05 -149580.667147
2015-10-06 -181081.404951
2015-10-07 109746.553965
2015-10-08 -3953.536012
2015-10-09 82929.908847
2015-10-10 152875.055086
2015-10-11 -4344.572382
2015-10-12 -201955.489539
2015-10-13 -251599.176155
2015-10-14 -196651.093380
2015-10-15 -249020.374452
2015-10-16 -2031.068979
Name: resid, Length: 1004, dtype: float64
Intercept for Recons Daily Reve LM is: -579.694334994291
Coeffcients for Recons Daily Reve LM is: [-5827.49422526]
Intercept for Daily Iterm Count LM is: -0.16337775403750426
Coeffcients for Daily Iterm Count LM is: [-10.09937909]
R-square for Daily Reve LM is: 0.006816656744495497
R-square for Daily Iterm Count LM is: 0.007708091856677758

Then, we will proceed with time series forecast with the Facebook Prophet to see how it goes

https://facebook.github.io/prophet/docs/quick_start.html#python-api

Implement Facebook Prophet for daily temp

Note we use first 30 months of data, from begining to 2015-06-30, [0:911] as the training set, and from 2015-07-01 to 2015-10-30 [911:1033] as the validation/test set

In [38]:
#date      DailyRev  ItemsSold   AvgTemp

df_fb_daytemp = pd.concat([df_MSC_SalesTemp.date[0:911], df_MSC_SalesTemp.AvgTemp[0:911]], axis=1)
df_fb_daytemp.columns = ['ds', 'y']

df_val_daytemp = pd.concat([df_MSC_SalesTemp.date[911:1033], df_MSC_SalesTemp.AvgTemp[911:1033]], axis=1)
df_val_daytemp.columns = ['ds', 'y_act']

true_daytemp = df_val_daytemp['y_act']

print(true_daytemp)
#print(df_fb_montemp.head())
#print(df_fb_montemp.shape)

#print(df_fb_montemp.ds)

m_temp = Prophet()

m_temp.fit(df_fb_daytemp)


date
2015-07-01 20.347826
2015-07-02 19.653846
2015-07-03 17.461538
2015-07-04 24.393939
2015-07-05 21.888889
2015-07-06 15.405405
2015-07-07 16.425000
2015-07-08 19.138889
2015-07-09 21.975000
2015-07-10 19.342105
2015-07-11 15.256410
2015-07-12 14.382353
2015-07-13 12.000000
2015-07-14 14.461538
2015-07-15 14.777778
2015-07-16 15.937500
2015-07-17 16.200000
2015-07-18 16.461538
2015-07-19 17.650000
2015-07-20 16.100000
2015-07-21 14.444444
2015-07-22 17.216216
2015-07-23 17.406250
2015-07-24 19.027027
2015-07-25 17.771429
2015-07-26 21.000000
2015-07-27 19.657895
2015-07-28 20.450000
2015-07-29 19.722222
2015-07-30 18.850000
 ... 
2015-10-01 7.763158
2015-10-02 11.307692
2015-10-03 12.052632
2015-10-04 8.390244
2015-10-05 11.170732
2015-10-06 4.878049
2015-10-07 0.142857
2015-10-08 0.970588
2015-10-09 -0.764706
2015-10-10 1.000000
2015-10-11 1.926829
2015-10-12 3.756098
2015-10-13 3.781250
2015-10-14 2.454545
2015-10-15 5.464286
2015-10-16 3.800000
2015-10-17 4.365854
2015-10-18 6.571429
2015-10-19 5.071429
2015-10-20 0.411765
2015-10-21 -1.058824
2015-10-22 0.966667
2015-10-23 2.088235
2015-10-24 4.048780
2015-10-25 5.285714
2015-10-26 5.428571
2015-10-27 4.540541
2015-10-28 1.128205
2015-10-29 0.151515
2015-10-30 0.529412
Name: y_act, Length: 122, dtype: float64
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Out[24]: <fbprophet.forecaster.Prophet at 0x7f8c10c21978>

In [39]:
future_temp = m_temp.make_future_dataframe(periods = 180)  

forecast_temp = m_temp.predict(future_temp)
m_temp.plot(forecast_temp)

display()



In [40]:
m_temp.plot_components(forecast_temp)

display()



In [41]:
# Check the y-hats for daily temp that will be used to calculate RMSE

print(forecast_temp[['ds','yhat']][911:1033])

pred_temp = forecast_temp['yhat'][911:1033]
print(pred_temp)



ds yhat
911 2015-07-01 18.955885
912 2015-07-02 19.133728
913 2015-07-03 19.418582
914 2015-07-04 19.599060
915 2015-07-05 19.698393
916 2015-07-06 19.708755
917 2015-07-07 19.554282
918 2015-07-08 19.462414
919 2015-07-09 19.438127
920 2015-07-10 19.526111
921 2015-07-11 19.520725
922 2015-07-12 19.450569
923 2015-07-13 19.312662
924 2015-07-14 19.035317
925 2015-07-15 18.849370
926 2015-07-16 18.762318
927 2015-07-17 18.820419
928 2015-07-18 18.818610
929 2015-07-19 18.785059
930 2015-07-20 18.715355
931 2015-07-21 18.535424
932 2015-07-22 18.472819
933 2015-07-23 18.530954
934 2015-07-24 18.751316
935 2015-07-25 18.923513
936 2015-07-26 19.069974
937 2015-07-27 19.180301
938 2015-07-28 19.174344
939 2015-07-29 19.273665
940 2015-07-30 19.475939
... ... ...
1003 2015-10-01 6.441271
1004 2015-10-02 6.216333
1005 2015-10-03 5.926672
1006 2015-10-04 5.600344
1007 2015-10-05 5.233149
1008 2015-10-06 4.751584
1009 2015-10-07 4.384157
1010 2015-10-08 4.135639
1011 2015-10-09 4.049268
1012 2015-10-10 3.916800
1013 2015-10-11 3.763181
1014 2015-10-12 3.580861
1015 2015-10-13 3.292828
1016 2015-10-14 3.124008
1017 2015-10-15 3.075594
1018 2015-10-16 3.187336
1019 2015-10-17 3.247650
1020 2015-10-18 3.278359
1021 2015-10-19 3.269056
1022 2015-10-20 3.140182
1023 2015-10-21 3.114461
1024 2015-10-22 3.191243
1025 2015-10-23 3.408807
1026 2015-10-24 3.554474
1027 2015-10-25 3.649337
1028 2015-10-26 3.682610
1029 2015-10-27 3.574679
1030 2015-10-28 3.548511
1031 2015-10-29 3.603967
1032 2015-10-30 3.780071

[122 rows x 2 columns]
911 18.955885
912 19.133728
913 19.418582
914 19.599060
915 19.698393
916 19.708755
917 19.554282
918 19.462414
919 19.438127
920 19.526111
921 19.520725
922 19.450569
923 19.312662
924 19.035317
925 18.849370
926 18.762318
927 18.820419
928 18.818610
929 18.785059
930 18.715355
931 18.535424
932 18.472819
933 18.530954
934 18.751316
935 18.923513
936 19.069974
937 19.180301
938 19.174344
939 19.273665
940 19.475939
 ... 
1003 6.441271
1004 6.216333
1005 5.926672
1006 5.600344
1007 5.233149
1008 4.751584
1009 4.384157
1010 4.135639
1011 4.049268
1012 3.916800
1013 3.763181
1014 3.580861
1015 3.292828
1016 3.124008
1017 3.075594
1018 3.187336
1019 3.247650
1020 3.278359
1021 3.269056
1022 3.140182
1023 3.114461
1024 3.191243
1025 3.408807
1026 3.554474
1027 3.649337
1028 3.682610
1029 3.574679
1030 3.548511
1031 3.603967
1032 3.780071
Name: yhat, Length: 122, dtype: float64

Implement Facebook Prophet for daily revenue

In [43]:


#date      DailyRev  ItemsSold   AvgTemp

df_fb_dayre = pd.concat([df_MSC_SalesTemp.date[0:911], df_MSC_SalesTemp.DailyRev[0:911]], axis=1)
df_fb_dayre.columns = ['ds', 'y']

df_val_dayre = pd.concat([df_MSC_SalesTemp.date[911:1033], df_MSC_SalesTemp.DailyRev[911:1033]], axis=1)
df_val_dayre.columns = ['ds', 'y_act']

true_dayre = df_val_dayre['y_act']

print(true_dayre)
#print(df_fb_montemp.head())
#print(df_fb_montemp.shape)

#print(df_fb_montemp.ds)

m_dr = Prophet()

m_dr.fit(df_fb_dayre)




date
2015-07-01 5.398558e+05
2015-07-02 5.456171e+05
2015-07-03 6.506790e+05
2015-07-04 6.284578e+05
2015-07-05 6.159409e+05
2015-07-06 5.569292e+05
2015-07-07 6.855317e+05
2015-07-08 5.576171e+05
2015-07-09 5.809675e+05
2015-07-10 6.707202e+05
2015-07-11 8.348605e+05
2015-07-12 6.027129e+05
2015-07-13 5.269021e+05
2015-07-14 5.449590e+05
2015-07-15 5.852149e+05
2015-07-16 5.159575e+05
2015-07-17 5.914194e+05
2015-07-18 6.318247e+05
2015-07-19 5.644720e+05
2015-07-20 5.429241e+05
2015-07-21 5.323281e+05
2015-07-22 4.828196e+05
2015-07-23 6.461980e+05
2015-07-24 4.814181e+05
2015-07-25 5.366773e+05
2015-07-26 4.448990e+05
2015-07-27 3.682480e+05
2015-07-28 3.849833e+05
2015-07-29 4.283750e+05
2015-07-30 4.707147e+05
 ... 
2015-10-01 7.586883e+05
2015-10-02 8.827069e+05
2015-10-03 1.258566e+06
2015-10-04 1.007520e+06
2015-10-05 4.497220e+05
2015-10-06 3.542838e+05
2015-10-07 6.528473e+05
2015-10-08 5.577432e+05
2015-10-09 6.911870e+05
2015-10-10 7.784652e+05
2015-10-11 6.272619e+05
2015-10-12 4.279103e+05
2015-10-13 4.008448e+05
2015-10-14 4.411201e+05
2015-10-15 3.855807e+05
2015-10-16 6.860915e+05
2015-10-17 7.062577e+05
2015-10-18 5.400470e+05
2015-10-19 4.298047e+05
2015-10-20 6.681265e+05
2015-10-21 5.550158e+05
2015-10-22 5.755465e+05
2015-10-23 9.626206e+05
2015-10-24 9.814669e+05
2015-10-25 7.941041e+05
2015-10-26 5.919920e+05
2015-10-27 4.912220e+05
2015-10-28 4.517047e+05
2015-10-29 5.294738e+05
2015-10-30 7.930866e+05
Name: y_act, Length: 122, dtype: float64
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Out[28]: <fbprophet.forecaster.Prophet at 0x7f8c10d55e48>

In [44]:

future_dr = m_dr.make_future_dataframe(periods = 180)  

forecast_dr = m_dr.predict(future_dr)
m_dr.plot(forecast_dr)

display()


In [45]:
m_dr.plot_components(forecast_dr)

display()



In [46]:
# Check the y-hats for monthly revenue that will be used to calculate RMSE

print(forecast_dr[['ds','yhat']][911:1033])

pred_dr = forecast_dr['yhat'][911:1033]
print(pred_dr)


ds yhat
911 2015-07-01 4.609794e+05
912 2015-07-02 4.848492e+05
913 2015-07-03 6.741888e+05
914 2015-07-04 9.022196e+05
915 2015-07-05 6.939356e+05
916 2015-07-06 4.007732e+05
917 2015-07-07 4.193895e+05
918 2015-07-08 4.215246e+05
919 2015-07-09 4.447912e+05
920 2015-07-10 6.341166e+05
921 2015-07-11 8.627303e+05
922 2015-07-12 6.556153e+05
923 2015-07-13 3.641785e+05
924 2015-07-14 3.850307e+05
925 2015-07-15 3.898500e+05
926 2015-07-16 4.161743e+05
927 2015-07-17 6.088456e+05
928 2015-07-18 8.410010e+05
929 2015-07-19 6.375269e+05
930 2015-07-20 3.497335e+05
931 2015-07-21 3.741374e+05
932 2015-07-22 3.823292e+05
933 2015-07-23 4.117680e+05
934 2015-07-24 6.072294e+05
935 2015-07-25 8.417981e+05
936 2015-07-26 6.403242e+05
937 2015-07-27 3.540983e+05
938 2015-07-28 3.796353e+05
939 2015-07-29 3.885412e+05
940 2015-07-30 4.183073e+05
... ... ...
1003 2015-10-01 6.102580e+05
1004 2015-10-02 8.119641e+05
1005 2015-10-03 1.051002e+06
1006 2015-10-04 8.520704e+05
1007 2015-10-05 5.663441e+05
1008 2015-10-06 5.902572e+05
1009 2015-10-07 5.953762e+05
1010 2015-10-08 6.191964e+05
1011 2015-10-09 8.065897e+05
1012 2015-10-10 1.030796e+06
1013 2015-10-11 8.168777e+05
1014 2015-10-12 5.163895e+05
1015 2015-10-13 5.261569e+05
1016 2015-10-14 5.181363e+05
1017 2015-10-15 5.302005e+05
1018 2015-10-16 7.075767e+05
1019 2015-10-17 9.238264e+05
1020 2015-10-18 7.042903e+05
1021 2015-10-19 4.007492e+05
1022 2015-10-20 4.101943e+05
1023 2015-10-21 4.046811e+05
1024 2015-10-22 4.221101e+05
1025 2015-10-23 6.076618e+05
1026 2015-10-24 8.347759e+05
1027 2015-10-25 6.285968e+05
1028 2015-10-26 3.406383e+05
1029 2015-10-27 3.675588e+05
1030 2015-10-28 3.810224e+05
1031 2015-10-29 4.184890e+05
1032 2015-10-30 6.246592e+05

[122 rows x 2 columns]
911 4.609794e+05
912 4.848492e+05
913 6.741888e+05
914 9.022196e+05
915 6.939356e+05
916 4.007732e+05
917 4.193895e+05
918 4.215246e+05
919 4.447912e+05
920 6.341166e+05
921 8.627303e+05
922 6.556153e+05
923 3.641785e+05
924 3.850307e+05
925 3.898500e+05
926 4.161743e+05
927 6.088456e+05
928 8.410010e+05
929 6.375269e+05
930 3.497335e+05
931 3.741374e+05
932 3.823292e+05
933 4.117680e+05
934 6.072294e+05
935 8.417981e+05
936 6.403242e+05
937 3.540983e+05
938 3.796353e+05
939 3.885412e+05
940 4.183073e+05
 ... 
1003 6.102580e+05
1004 8.119641e+05
1005 1.051002e+06
1006 8.520704e+05
1007 5.663441e+05
1008 5.902572e+05
1009 5.953762e+05
1010 6.191964e+05
1011 8.065897e+05
1012 1.030796e+06
1013 8.168777e+05
1014 5.163895e+05
1015 5.261569e+05
1016 5.181363e+05
1017 5.302005e+05
1018 7.075767e+05
1019 9.238264e+05
1020 7.042903e+05
1021 4.007492e+05
1022 4.101943e+05
1023 4.046811e+05
1024 4.221101e+05
1025 6.076618e+05
1026 8.347759e+05
1027 6.285968e+05
1028 3.406383e+05
1029 3.675588e+05
1030 3.810224e+05
1031 4.184890e+05
1032 6.246592e+05
Name: yhat, Length: 122, dtype: float64

In [47]:
from sklearn.metrics import mean_squared_error

mse_temp = mean_squared_error(true_daytemp, pred_temp)
rmse_temp = mse_temp**(0.5)

mse_dr = mean_squared_error(true_dayre, pred_dr)
rmse_dr = mse_dr**(0.5)

print('The RMSE for Daily Temperature is:', rmse_temp)

print('The RMSE for Daily Revenue is:', rmse_dr)



The RMSE for Daily Temperature is: 3.598394550747085
The RMSE for Daily Revenue is: 157820.8835738632

The RMSE for Daily Temperature is: 3.598394550747085  
The RMSE for Daily Revenue is: 157820.8835738632  

From Data Preparation Notebook, we know the following means and standard deviations:

Means:  
DailyRev:     824420.868021  
ItemsSold:      1234.406190  
AvgTemp:           6.950757  
 
Standard Deviations:   
DailyRev:     386737.832552  
ItemsSold:       570.724269  
AvgTemp:          10.367651  

In this case, the RMSEs for daily temperature and revenue as predicted by the Facebook Prophet is not that bad, but not great either.